Create a web scraping script that can extract Product related information from any 2 e-commerce websites such as product names, product category and other product details along with price.

In [ ]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
## import GST mapping rate mapping from gst file

gst_df =pd.read_csv("gst_mapping.csv")
amount = r"(?:₹\d+.\d+)"
#defining product 
class Product:
    def __init__(self,**kwargs):
        self.name=kwargs.get("name")
        self.image=kwargs.get("image")
        self.price = kwargs.get("price")
        
    class DefaultProduct:
        def __init__(self,**kwargs):
            self.name =kwargs.get("name")
            self.image-kwargs.get("image")
            
            
# define function to scrape products  information from website

def get_search_url(search_term):
  #generate url for each term"""
    search_term =search_term.replace(' ','+')
    url_template =f"https://www.flipkart.com/search?q={search_term}"
    return url_template

def filterProductName(item):
    product_name = ""
    a_title =[x.get("title") for x in item.find_all("a") if x.get("title") is not None]
    if item.find(class_="_4rR01T"):
        product_name = item.find(class_="_4rR01T").text
    
    elif any(a_title):
        product_name =a_title[0]
        
    return product_name

def category_name(soup):
    category = ""
    category_by =[x.get("title") for x in soup.find_all("a") if x.get("title") is not None]
    if soup.find(class_="_2qvBBJ _2Mji8F"):
        category = soup.find(class_="_2qvBBJ _2Mji8F").text
    elif any(category_by):
        category=category_by[0]
        
    return category
    


def search_products(search_term):
    url =get_search_url(search_term)
    #scraps Searched data from given url
    #sending GET requeste to website 
    response  = requests.get(url)
    #Parse HTML content
    soup = BeautifulSoup(response.content,"html.parser")
    #initializing trhe empty list to steore product information
    product_container = soup.find_all(class_="_13oc-S")
    #loop through all the products elements on webpage
    product_names =[]
    product_image=[]
    product_prices= []
   
    
    # Extract product  name, category and details :
    for item in product_container:
        ## adding product name in list 
        product_names.append(filterProductName(item)) 
        
        
        ## adding productimage in list 
        product_image.append(item.find("img")["src"])
        
         ## adding product price in list 
        product_prices.append(re.search(amount,str(item.find_all('div'))).group(0))
        
    #craeting dataframe
    df_dict = {"name":product_names,
            "image":product_image,
            "price":product_prices,
              "category":category}
    
    df =pd.DataFrame.from_dict(df_dict)
    return df
##merging gst information with product information
product_df =pd.merge(df,gst_df, on= category)
#wxporting to csv 
product_df.tocsv("product_info.csv", index =False)